In [1]:
import pathlib
import json
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
# !pip install folium
import folium

from pathlib import Path

from collections import Counter

## Analyse data

In [2]:
df_sum = None
for df_path in tqdm(list(Path("../data/local/tables/").iterdir())[-10:]):
    if df_sum is None: 
        df_sum = pd.read_csv(df_path, encoding='utf-8', low_memory=False)
    else : 
        df_sum.append(pd.read_csv(df_path, encoding='utf-8', low_memory=False))

100%|██████████| 10/10 [00:23<00:00,  2.34s/it]


In [3]:
df_sum[df_sum.duplicated()]

,id,imei,name,stateCode,stateName,lat,lng,speed,orientation,gpstime,routeId,routeName,routeColour,inDepo,busNumber,perevId,perevName,remark,online,idBusTypes


In [4]:
df_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280232 entries, 0 to 280231
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           280232 non-null  int64  
 1   imei         280232 non-null  int64  
 2   name         280232 non-null  object 
 3   stateCode    280232 non-null  object 
 4   stateName    280232 non-null  object 
 5   lat          280232 non-null  float64
 6   lng          280232 non-null  float64
 7   speed        280232 non-null  float64
 8   orientation  280232 non-null  float64
 9   gpstime      280232 non-null  object 
 10  routeId      280232 non-null  int64  
 11  routeName    280232 non-null  object 
 12  routeColour  280232 non-null  object 
 13  inDepo       280232 non-null  bool   
 14  busNumber    280232 non-null  object 
 15  perevId      280232 non-null  int64  
 16  perevName    280232 non-null  object 
 17  remark       280232 non-null  object 
 18  online       280232 non-

### SQL Shema bouild

In [7]:
df_sum['inDepo'].value_counts()

False    271876
True       8356
Name: inDepo, dtype: int64

In [61]:
# df_sum[['routeId', 'routeName', 'routeColour']].value_counts()
# df_sum[['imei', 'name', 'busNumber', 'perevId']].value_counts()
df_sum[['idBusTypes']].value_counts()

idBusTypes
1             200704
2              79528
dtype: int64

In [63]:
df_sum[['imei', 'name', 'busNumber', 'perevId', 'idBusTypes']].value_counts()

imei             name  busNumber  perevId  idBusTypes
355227045371655  A1    6513       7        1             5521
355227046638821  A211  1181       13       1             5328
355228042084283  A207  1032       13       1             5239
355227045540192  A91   7762       1        1             4498
355227044854750  A22   7665       1        1             4483
                                                         ... 
355228043199080  H103  8716       12       1               18
355227045539491  Am86  1027       2        1               10
355228043162351  H107  9138       12       1                5
355227045600855  A142  377        6        2                2
355227046447371  H65   355        6        2                1
Length: 98, dtype: int64

In [42]:
a = df_sum[['remark']].value_counts()

In [57]:
df_sum[df_sum['imei'] == 355227045541257][['imei', 'name', 'busNumber', 'remark']].value_counts()


imei             name  busNumber  remark    
355227045541257  A57   9043       0325 DNSNK    2007
                                  9043 DNSNK    1870
dtype: int64

In [53]:
from collections import Counter

In [58]:
# Counter([k[0] for k in a.to_dict()])

In [59]:
# a.to_dict()

0   id           280232 non-null  int64  

### USELESS ?!
 3   stateCode    280232 non-null  object #only value is 'used'
 
 4   stateName    280232 non-null  object #only value is 'used'
 
 18  online       280232 non-null  bool   #only value is 'True'
 

### ROUTES
 10  routeId      280232 non-null  int64  
 
 11  routeName    280232 non-null  object 
 
 12  routeColour  280232 non-null  object 
    

### OWNER
 15  perevId      280232 non-null  int64  
 
 16  perevName    280232 non-null  object 


### WEEHCLE 
 1   imei         280232 non-null  int64  
 
 2   name         280232 non-null  object 
 
 14  busNumber    280232 non-null  object 
 
 17  remark       280232 non-null  object 
 
 19  idBusTypes   280232 non-null  int64  

### GPS
 5   lat          280232 non-null  float64
 
 6   lng          280232 non-null  float64
 
 7   speed        280232 non-null  float64
 
 8   orientation  280232 non-null  float64
 
 9   gpstime      280232 non-null  object 
 
13  inDepo       280232 non-null  bool   


## CODE 
```
Table rowdata.gpsdata {
id int [pk, increment]
lat float
lng float
speed float
orientation float
gpstime datetime
inDepo bool

routeId int [ref: > E.id]

response_datetime datetime

edited datetime

Indexes {
(id) [pk]
}
}

Table rowdata.vehicle as E {
id int [pk, increment]
imei int
name varchar
busNumber int
remark varchar

perevId int [ref: > O.id]
routeId int [ref: > R.id]

edited datetime

Indexes {
(id) [pk]
}
}

Table rowdata.owner as O {
id int [pk, increment]
perevName varchar
edited datetime

Indexes {
(id) [pk]
}
}

Table rowdata.route as R {
id int [pk, increment]
routeName varchar
routeColour varchar

etited datetime

Indexes {
(id) [pk]
}
}
```

In [3]:
import psycopg2
import os

In [4]:
conn = psycopg2.connect(
    host=os.environ['RDS_HOSTNAME'],
    database="pteta_db",
    user="postgres",
    password=os.environ['RDS_PTETA_DB_PASSWORD'])

In [5]:
with conn.cursor() as cursor: 
    cursor.execute("select * from information_schema.tables")
    print(cursor.fetchall())


[('pteta_db', 'pteta', 'gpsdata', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('pteta_db', 'pteta', 'owner', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('pteta_db', 'pteta', 'route', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('pteta_db', 'pg_catalog', 'pg_type', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('pteta_db', 'pg_catalog', 'pg_foreign_table', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('pteta_db', 'pteta', 'vehicle', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('pteta_db', 'pg_catalog', 'pg_roles', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('pteta_db', 'pg_catalog', 'pg_settings', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('pteta_db', 'pg_catalog', 'pg_prepared_statements', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('pteta_db', 'pg_catalog', 'pg_stat_progress_vacuum', 'VIEW', None, None, None, None, None, 'NO',

In [6]:
with conn.cursor() as cursor: 
    cursor.execute("SELECT * FROM pteta.owner;")
    print(cursor.fetchall())

[(6, 'ЧТУ', datetime.datetime(2022, 12, 1, 22, 34, 31, 931733)), (12, 'Vitaliy2019', datetime.datetime(2022, 12, 1, 23, 7, 12, 328328)), (7, 'Віталій / Олег Бойко ', datetime.datetime(2022, 12, 1, 23, 7, 12, 526166)), (1, 'Денисівка', datetime.datetime(2022, 12, 1, 23, 7, 12, 680715)), (13, 'Олег Валентинович 39 марш', datetime.datetime(2022, 12, 1, 23, 7, 12, 844328)), (2, 'Тарас', datetime.datetime(2022, 12, 1, 23, 7, 13, 626))]


## TransportOperator

In [6]:
from PTETA.utils.transport.TransportOperator import TransportOperator

In [8]:
?TransportOperator.get_table

Signature: TransportOperator.get_table(connection: psycopg2.extensions.connection) -> List[ForwardRef('TransportOperator')]
Docstring: <no docstring>
File:      d:\projects\pteta\pteta\utils\transport\transportoperator.py
Type:      method


In [7]:
TransportOperator.get_table(conn)

[TransportOperator(id=6, name='ЧТУ'),
 TransportOperator(id=12, name='Vitaliy2019'),
 TransportOperator(id=7, name='Віталій / Олег Бойко '),
 TransportOperator(id=1, name='Денисівка'),
 TransportOperator(id=13, name='Олег Валентинович 39 марш'),
 TransportOperator(id=2, name='Тарас')]

In [21]:
for row in  df_sum[["perevId", "perevName"]].drop_duplicates().values:
    trans_operator = TransportOperator(*row)
    
    if (not trans_operator.is_in_table(conn)): 
        trans_operator.insert_in_table(conn)

In [22]:
df_sum[["perevId", "perevName"]].drop_duplicates().values

array([[6, 'ЧТУ'],
       [12, 'Vitaliy2019'],
       [7, 'Віталій / Олег Бойко '],
       [1, 'Денисівка'],
       [13, 'Олег Валентинович 39 марш'],
       [2, 'Тарас']], dtype=object)

### ROUTES

10 routeId 280232 non-null int64

11 routeName 280232 non-null object

12 routeColour

In [30]:
from PTETA.utils.transport.TransportRoute import TransportRoute

In [20]:
import importlib
import PTETA
importlib.reload(PTETA)

<module 'PTETA' from 'D:\\projects\\PTETA\\PTETA\\__init__.py'>

In [97]:
df_sum[["routeId", "routeName", "routeColour"]].drop_duplicates().values

array([[37, 'T', 'coral'],
       [31, '6/6a', 'deeppink'],
       [21, '38', 'deeppink'],
       [41, '10A', 'black'],
       [20, 'A', 'navy'],
       [42, '39', 'coral'],
       [23, '19', 'teal'],
       [19, '10', 'black'],
       [11, '9', 'magenta'],
       [2, '11', 'green'],
       [6, '4', 'magenta'],
       [27, '3/3a', 'green'],
       [16, '2', 'green'],
       [4, '5', 'orange'],
       [3, '12', 'blue'],
       [12, '20', 'maroon'],
       [7, '6', 'sienna'],
       [10, '34', 'navy'],
       [45, '1', 'navy'],
       [9, '27', 'black'],
       [38, '1T', 'grey'],
       [49, '2A', 'red'],
       [5, '13', 'brown'],
       [33, '8T', 'coral'],
       [17, '23', 'red'],
       [48, '1A', 'olive'],
       [14, '9A', 'magenta'],
       [26, '4T', 'purple'],
       [25, '2T', 'olive'],
       [39, '5T', 'maroon'],
       [34, '11/3', 'teal']], dtype=object)

In [26]:
TransportRoute(1, 'a', 'routeColour').is_in_table(conn)

False

In [ ]:
TransportRoute.insert_in_table

In [36]:
for row in df_sum[["routeId", "routeName", "routeColour"]].drop_duplicates().values:
    trans_route = TransportRoute(*row)
    
    if (not trans_route.is_in_table(conn)): 
        trans_route.insert_in_table(conn)
        
TransportRoute.get_table(conn)

TypeError: get_table() missing 1 required positional argument: 'connection'

### WEEHCLE
1 imei 280232 non-null int64

2 name 280232 non-null object

14 busNumber 280232 non-null object

17 remark 280232 non-null object

19 idBusTypes 280232 non-null int64

In [130]:
# df_sum.head(3)

In [7]:
dt = df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].drop_duplicates().values
dt.shape

(148, 6)

In [132]:
# df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].value_counts()

In [129]:
# len(set(dt[:, 3]))

99

In [133]:
# "imei", "name", "busNumber", "remark" (99) ->  "perevId", "routeId" = 148

In [8]:
from PTETA.utils.transport.TransportVehicle import TransportVehicle

In [21]:
import importlib
import PTETA
importlib.reload(PTETA)


<module 'PTETA' from 'D:\\projects\\PTETA\\PTETA\\__init__.py'>

In [152]:
dt = df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].drop_duplicates().values
dt[:3, :]

array([[355227045600830, 'A178', '310', 'Тролейбус 310 DNSNK', 6, 37],
       [355227046451662, 'H76', '350', 'Тролейбус 350 DNSNTNK', 6, 31],
       [355227045369527, 'A6', '3557', '3557 DNSNK ', 12, 21]],
      dtype=object)

In [13]:
trans_vehicle = TransportVehicle(None, 355227045369527, 'A6', '3557', '3557 DNSNK ', 12, 21)
trans_vehicle.is_in_table(conn)
# isTransportVehicleInTable(conn, trans_vehicle)

True

In [11]:
# TransportVehicle.get_table(conn)

In [12]:
dt = df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].drop_duplicates().values

for row in tqdm(dt):
    trans_vehicle = TransportVehicle(None, *row)
    
    if (not trans_vehicle.is_in_table(conn)): 
        trans_vehicle.insert_in_table(conn)
        
TransportVehicle.get_table(conn)

100%|██████████| 148/148 [00:06<00:00, 22.57it/s]


[TransportVehicle(id=1, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37),
 TransportVehicle(id=2, imei='355227046451662', name='H76', busNumber='350', remark='Тролейбус 350 DNSNTNK', perevId=6, routeId=31),
 TransportVehicle(id=3, imei='355227045369527', name='A6', busNumber='3557', remark='3557 DNSNK ', perevId=12, routeId=21),
 TransportVehicle(id=4, imei='355227045540176', name='A83', busNumber='5150', remark=' 5150 DNS', perevId=7, routeId=41),
 TransportVehicle(id=5, imei='355227046453387', name='H75', busNumber='3627', remark='3627 DNSNTNK', perevId=1, routeId=20),
 TransportVehicle(id=6, imei='355228042084283', name='A207', busNumber='1032', remark='1032 DNSNK', perevId=13, routeId=42),
 TransportVehicle(id=7, imei='355227045371655', name='A1', busNumber='6513', remark='6513 DNSNK', perevId=7, routeId=23),
 TransportVehicle(id=8, imei='355227046451407', name='H68', busNumber='0855', remark='0855 DNSNTNK', perevId=13, rout

In [14]:
# connection_config
# host=os.environ['RDS_HOSTNAME'],
# database="pteta_db",
# user="postgres",
# password=os.environ['RDS_PTETA_DB_PASSWORD']

from PTETA.utils.transport.TransportOperator import TransportOperator
from PTETA.utils.transport.TransportRoute import TransportRoute
from PTETA.utils.transport.TransportVehicle import TransportVehicle

class Monitor:
    
    db_connection: Connection = None
        
    operator_to_id : dict = dict()
    id_to_operator : dict = dict()
        
    route_to_id : dict = dict()
    id_to_route : dict = dict()

    vehicle_to_id : dict = dict()
    id_to_vehicle : dict = dict()
    
        
    def __init__(self, connection_config: dict, **kwarg: dict): 
        self.db_connection = psycopg2.connect(**connection_config)
        
        operator_list = TransportOperator.get_table(self.db_connection)
        self.operator_to_id = dict({operator : operator.id for operator in operator_list})
        self.id_to_operator = dict({operator.id : operator for operator in operator_list})
        
        route_list = TransportRoute.get_table(self.db_connection)
        self.route_to_id = dict({route : route.id for route in route_list})
        self.id_to_route = dict({route.id : route for route in route_list})
    
        vehicle_list = TransportVehicle.get_table(self.db_connection)
        self.vehicle_to_id = dict({vehicle : vehicle.id for vehicle in vehicle_list})
        self.id_to_vehicle = dict({vehicle.id : vehicle for vehicle in vehicle_list})
        
        
        REQUEST_URI
        
        
        
    def request_data():
        dt_now = datetime.now()

        try:
            request = requests.get(REQUEST_URI)
            if (request is None) or (request.text is None):
                return
            response_cur = json.loads(request.text)

            global response_prev

            keys_prev = set(response_prev.keys())
            keys_cur = set(response_cur.keys())

            optimized_data_list = list()
            for imei in keys_prev.intersection(keys_cur):
                if response_prev[imei] != response_cur[imei]:
                    #response_cur[imei]['response_datetime']
                    optimized_data_list += [response_cur[imei]]

            for imei in keys_cur.difference(keys_prev):
                optimized_data_list += [response_cur[imei]]

            if optimized_data_list:
                path = f"trans_data_{dt_now.strftime('%d_%b_%Y').upper()}"
                pathlib.Path(path).mkdir(parents=True, exist_ok=True)

                f_name = pathlib.Path(f"trans_{dt_now.strftime('%Y-%m-%d %H;%M;%S')}.json")

                result = {d['imei']: d for d in optimized_data_list}
                if (path/f_name).is_file():
                    print(f"{dt_now.strftime('%Y-%m-%d %H;%M;%S')} : '{f_name}' file is already exists!")
                    return

                with open(str(path/f_name), 'w', encoding='utf8') as out_f:
                    json.dump(result, out_f, ensure_ascii=False)

                response_prev = response_cur
        except (requests.Timeout, requests.ConnectionError, requests.HTTPError) as err:
            print(f"{dt_now.strftime('%Y-%m-%d %H;%M;%S')} : error while trying to GET data\n"
                  f"\t{err}\n")

        
        
        
        
        
        
    
    

NameError: name 'Connection' is not defined

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import json
import requests
import os
import pandas as pd
import pathlib
import shutil
import time
from tqdm.auto import tqdm
import zipfile

# from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

REQUEST_URI = 'http://www.trans-gps.cv.ua/map/tracker/?selectedRoutesStr='
END_DATE= '2022-12-25 23:59:00'
START_DATE = '2022-10-03 00:03:00'
REQ_TIME_DELTA = 1.1

response_prev = dict()


def request_data():
    dt_now = datetime.now()

    try:
        request = requests.get(REQUEST_URI)
        if (request is None) or (request.text is None):
            return
        response_cur = json.loads(request.text)

        global response_prev

        keys_prev = set(response_prev.keys())
        keys_cur = set(response_cur.keys())

        optimized_data_list = list()
        for imei in keys_prev.intersection(keys_cur):
            if response_prev[imei] != response_cur[imei]:
                #response_cur[imei]['response_datetime']
                optimized_data_list += [response_cur[imei]]

        for imei in keys_cur.difference(keys_prev):
            optimized_data_list += [response_cur[imei]]

        if optimized_data_list:
            path = f"trans_data_{dt_now.strftime('%d_%b_%Y').upper()}"
            pathlib.Path(path).mkdir(parents=True, exist_ok=True)

            f_name = pathlib.Path(f"trans_{dt_now.strftime('%Y-%m-%d %H;%M;%S')}.json")

            result = {d['imei']: d for d in optimized_data_list}
            if (path/f_name).is_file():
                print(f"{dt_now.strftime('%Y-%m-%d %H;%M;%S')} : '{f_name}' file is already exists!")
                return

            with open(str(path/f_name), 'w', encoding='utf8') as out_f:
                json.dump(result, out_f, ensure_ascii=False)

            response_prev = response_cur
    except (requests.Timeout, requests.ConnectionError, requests.HTTPError) as err:
        print(f"{dt_now.strftime('%Y-%m-%d %H;%M;%S')} : error while trying to GET data\n"
              f"\t{err}\n")


def read_json_TPE(f_path):
    try:
        with open(f_path, 'r', encoding='utf-8') as file:
            return (json.load(file), None)
    except Exception as e:
        print(f"Unable to open : {f_path}")
        print(f"Exception is '{e}'")
        return (None, file)


def get_df(jsons_content):
    data = list()
    for r in jsons_content:
        if r[1] is None:
            data += [val for _, val in r[0].items()]

    df = pd.DataFrame(data)
    duplicates_num = df.duplicated().value_counts().get(True, 0)
    print(f"Total duplicates are {duplicates_num} of {len(df)}")
    return df.drop_duplicates()


def folder_to_df(folder):
    fpaths_list = sorted(list(folder.iterdir()) )

    with ThreadPoolExecutor(max_workers=40) as executor:
        fpath_list = fpaths_list
        results = list(
            tqdm(
                executor.map(read_json_TPE, fpath_list),
                total=len(fpath_list), miniters=int(len(fpath_list) // 10),
                desc=f"Read {str(folder)} to write df", ascii=True
            )
        )

    df = get_df(results)
    df.to_csv(f"tables/data_for_{str(folder)[11:]}.csv",
              encoding='utf-8', index=False, header=True)


def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in tqdm(files, position=0, leave=True, desc=f"Zip {path}"):
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))


def write_zip_archive(folder):
    zip_fname = f"archives/{folder}.zip"
    with zipfile.ZipFile(zip_fname, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipdir(f"{folder}", zipf)


def afterprocess_data():
    print(f"Data aterprocessing started at {datetime.now()}")
    folders_list = [p for p in pathlib.Path().iterdir() if 'trans_data_' in str(p)]
    folders_list = sorted(folders_list, key=lambda x: datetime.strptime(str(x)[11:], '%d_%b_%Y'))[:-1]
    print(f"\tFolders to process {folders_list}")
    pbar = tqdm(folders_list, leave=False)
    for folder in pbar:
        pbar.set_postfix_str(str(folder))
        print(f"\tFolder conversion to df started at {datetime.now()}")
        folder_to_df(folder)
        print(f"\tFolder conversion to df fibished at {datetime.now()}")
        print(f"\tArchiving of folder started at {datetime.now()}")
        write_zip_archive(folder)
        print(f"\tArchiving of folder finished at {datetime.now()}")
        time.sleep(1)
        print(f"Start delete {folder}")
        shutil.rmtree(folder)
        print(f"{folder} is deleted")
    print(f"Data aterprocessing finished at {datetime.now()}")




def main():
    scheduler = BackgroundScheduler(job_defaults={'max_instances': 8})
    scheduler.add_job(
        request_data, 'interval',
        seconds=REQ_TIME_DELTA,
        end_date=END_DATE,
        id='listener')

    scheduler.add_job(
        afterprocess_data, 'interval',
        days=1,
        start_date=START_DATE,
        end_date=END_DATE,
        id='afterprocess_data'
    )

    print(datetime.now())
    while datetime.now().microsecond > 5_000:
        time.sleep(0.00002)
    print(datetime.now())
    scheduler.start()

    # This code will be executed after the sceduler has started
    try:
        print('Scheduler started!')
        display_files_num = True 
        prev_update = datetime.now().replace(minute=0, second=0, microsecond=0)
        while 1:
            dt_now = datetime.now()
            path = f"trans_data_{dt_now.strftime('%d_%b_%Y').upper()}"
            try:# for cases folder don't exist yet
                files_list = list(pathlib.Path(path).iterdir())
            except:
                files_list = []
            if len(files_list) % 1_000 > 10:
                display_files_num = True
            elif display_files_num:
                print(f"{dt_now.strftime('%Y-%m-%d %H:%M:%S')}"
                      f"\t Total files num is {len(files_list)}")
                display_files_num = False
                prev_update = dt_now

            if (dt_now - prev_update).seconds > 30 * 60:
                print(f"{dt_now.strftime('%Y-%m-%d %H:%M:%S')} : "
                      f"(last update {(dt_now - prev_update).seconds} sec ago)"
                      f"\t Total files num is {len(files_list)}")
                prev_update = dt_now

            time.sleep(3)
    except KeyboardInterrupt:
        if scheduler.state:
            scheduler.shutdown()


if __name__ == "__main__":
    main()


In [167]:
# sum_df.groupby('imei').size().reset_index(name='count').sort_values(['count'], ascending=False)